<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [104]:
import re

import spacy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import langid
from googletrans import Translator
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [42]:
nlp = spacy.load('en_core_web_lg')

In [43]:
yelp = pd.read_json('./data/review_sample.json', lines=True)

In [44]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [64]:
def validate_word(token):
    rules = set([
        token.is_punct,
        token.pos_ == 'SPACE',
        token.pos_ == 'PRON',
        token.pos_ == 'DET',
    ])
    
    return True not in rules

In [65]:
def tokenize(doc):
    return [token.lemma_ for token in nlp(doc) if validate_word(token)]

In [66]:
tokenize(u"The quick brown fox jumps over the lazy dog.  ")

['quick', 'brown', 'fox', 'jump', 'over', 'lazy', 'dog']

#### Part 1.5: Detect Language

In [25]:
reviews = yelp['text']
reviews.shape

(10000,)

In [26]:
yelp['lang'] = reviews.apply(langid.classify).str[0]

In [27]:
yelp['lang'].value_counts()

en    9994
lv       2
es       2
id       1
la       1
Name: lang, dtype: int64

In [28]:
non_en = yelp['lang'] != 'en'

In [29]:
non_en_values = yelp[non_en]['text'].str[:1500].values.tolist()

In [30]:
translator = Translator()
translations = translator.translate(non_en_values, dest='en')

In [31]:
yelp.loc[non_en, 'text'] = [translation.text for translation in translations]

In [32]:
reviews = yelp['text']
reviews.shape

(10000,)

In [33]:
yelp['lang'] = reviews.apply(langid.classify).str[0]

In [34]:
yelp['lang'].value_counts()

en    9994
lv       2
es       2
id       1
la       1
Name: lang, dtype: int64

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [69]:
tokens = reviews.apply(tokenize)

In [70]:
tokens[:20]

0     [beware, fake, fake, fake, also, own, small, b...
1     [come, here, for, lunch, Togo, service, be, qu...
2     [have, be, to, Vegas, dozen, of, time, and, ha...
3     [go, here, on, night, where, close, off, part,...
4     [3.5, to, 4, star, not, bad, for, price, $, 12...
5     [tasty, fast, casual, latin, street, food, men...
6     [show, be, absolutely, amazing, incredible, pr...
7     [come, for, Pho, and, really, enjoy, get, here...
8     [absolutely, most, unique, experience, in, nai...
9     [wow, walk, in, and, sit, at, bar, for, 10, mi...
10    [pop, in, for, dinner, yesterday, with, reserv...
11    [bad, stay, ever, so, first, end, up, pay, ove...
12    [great, friendly, customer, service, and, qual...
13    [food, be, great, be, super, busy, but, server...
14    [talk, about, get, rip, off, charge, $, 420, f...
15    [girl, night, out, tonight, with, kid, so, dec...
16    [stop, in, here, for, few, drink, fly, out, of...
17    [be, excellent, restaurant, and, encourage

In [71]:
vectorizer = TfidfVectorizer(tokenizer=tokenize)
X = vectorizer.fit_transform(reviews)

In [72]:
dtm = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
print(dtm.shape)
dtm.head()

(10000, 23022)


,"""eat","""hair","""was","""well",$,$2,$35,$5,','d,...,っ,ま,ゃ,ら,來一個熱辣辣牛肉粉，滿足,台湾鸡排,吃了太多tim,漁膳房,美味的味道,豆腐花
0,0.0,0.0,0.0,0.0,0.060491,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.081615,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.076258,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [78]:
fake_review = tokenize("Not the worst chinese food I've ever had. Forgot my fortune cookie.")
fake_review_vectorize = vectorizer.transform(fake_review)

In [79]:
neighbors = nn.kneighbors(fake_review_vectorize.todense())
neighbors

(array([[1.19006405, 1.19821941, 1.19852671, 1.19957979, 1.20184542,
         1.21323596, 1.21335749, 1.22101066, 1.22294512, 1.22881943],
        [0.79027074, 0.96005179, 1.01531882, 1.14849721, 1.1502259 ,
         1.17037217, 1.18080038, 1.18856006, 1.19919452, 1.20437573],
        [0.90218501, 0.94229529, 0.96870488, 1.01151261, 1.02513431,
         1.03706399, 1.04820767, 1.05968911, 1.09470385, 1.09881138],
        [1.10308972, 1.1200234 , 1.12245789, 1.12421956, 1.14637019,
         1.15120363, 1.15383282, 1.17548761, 1.18881519, 1.18957245],
        [1.09071518, 1.13800907, 1.15779532, 1.16888502, 1.17186158,
         1.17723281, 1.18352129, 1.18564181, 1.18889797, 1.18973561],
        [1.12000772, 1.12783248, 1.14500098, 1.16093364, 1.1724754 ,
         1.19327466, 1.19679312, 1.19770837, 1.2041125 , 1.22107604],
        [1.09071518, 1.13800907, 1.15779532, 1.16888502, 1.17186158,
         1.17723281, 1.18352129, 1.18564181, 1.18889797, 1.18973561],
        [1.16117518, 1.1695

In [80]:
reviews[neighbors[1][0]].values

array(["We really wanted to like this theater but it didn't happen.  The service was not good and the food was cold when it arrived.  Very disappointing and we will not be returning.",
       "For the people that said service was Excellent, I question your opinion in Excellence.\n   The food here is Pretty good if you're really into sushi. The Vampire roll is my fave and has garlic ponzu sauce on top, they have Great Udon and Miso. LOVE their Poke and seaweed salad and they have a few other good ones too AND out of the 6 times I've gone, I never had to wait for a table. \n   BIG But however, Absolutely Trust in these words and the others that say the service is not good and the woman that are waiting on you are mostly standing in the back doing nothing. I have not sat at the bar yet.  \n   \n   If you have somewhere to be, do not come here. It's the slowest sushi restaurant I've been to in LV and I've been to at least 10 in this city. Look forward to them bringing someone else's order 

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

Because spacy takes so long, i'm not going to use a pipeline, that will have to refit that every time, would take hours... sue me lol 🤷‍

In [81]:
y = yelp['stars']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [83]:
clf = LinearSVC(random_state=42)

In [84]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [85]:
y_pred = clf.predict(X_test)
np.mean(y_pred == y_test)

0.6084848484848485

In [86]:
params = {
    'penalty': ('l1', 'l2'),
    'loss': ('hinge', 'squared_hinge'),
    'C': (0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95),
}

In [87]:
gs_clf = GridSearchCV(clf, params, n_jobs = -1, cv=5, verbose=10)

In [88]:
gs_clf.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   28.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=42, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': (0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95),
                         'loss': ('hinge', 'squared_hinge'),
                         'penalty': ('l1', 'l2')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [89]:
gs_clf.best_score_

0.6222388059701492

In [90]:
gs_clf.best_params_

{'C': 0.2, 'loss': 'squared_hinge', 'penalty': 'l2'}

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [91]:
np.__version__

'1.18.1'

In [92]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [97]:
id2word = Dictionary(tokens.values.tolist())

Create a bag of words representation of the entire corpus

In [99]:
corpus = [id2word.doc2bow(text) for text in tokens.values]

Your LDA model should be ready for estimation: 

In [100]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

In [101]:
lda.print_topics()

[(0,
  '0.059*"be" + 0.048*"and" + 0.030*"to" + 0.020*"have" + 0.016*"not" + 0.015*"for" + 0.014*"of" + 0.012*"with" + 0.011*"in" + 0.010*"do"'),
 (1,
  '0.054*"be" + 0.042*"and" + 0.027*"to" + 0.021*"of" + 0.016*"in" + 0.016*"have" + 0.015*"not" + 0.014*"for" + 0.011*"but" + 0.009*"with"'),
 (2,
  '0.063*"be" + 0.044*"and" + 0.024*"of" + 0.024*"to" + 0.019*"have" + 0.018*"in" + 0.014*"not" + 0.014*"for" + 0.010*"with" + 0.010*"but"'),
 (3,
  '0.052*"be" + 0.047*"and" + 0.037*"to" + 0.021*"have" + 0.018*"for" + 0.014*"in" + 0.013*"not" + 0.012*"of" + 0.010*"with" + 0.010*"on"'),
 (4,
  '0.063*"be" + 0.041*"and" + 0.028*"to" + 0.021*"not" + 0.018*"of" + 0.017*"have" + 0.015*"do" + 0.013*"for" + 0.012*"in" + 0.010*"with"'),
 (5,
  '0.066*"be" + 0.039*"and" + 0.035*"to" + 0.022*"have" + 0.018*"not" + 0.018*"of" + 0.012*"in" + 0.011*"for" + 0.010*"on" + 0.009*"with"'),
 (6,
  '0.076*"be" + 0.040*"and" + 0.027*"have" + 0.026*"to" + 0.017*"of" + 0.016*"for" + 0.015*"not" + 0.012*"with" + 0.0

In [105]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [106]:
topics = [' '.join(t[0:5]) for t in words]

In [107]:
topics

['be and to have not',
 'be and to of in',
 'be and of to have',
 'be and to have for',
 'be and to not of',
 'be and to have not',
 'be and have to of',
 'be and to of not',
 'be and to have in',
 'be and to of have']

Create 1-2 visualizations of the results

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)